In [6]:
import openai
from openai.types.chat import ChatCompletionMessage
import json
import requests

MOVIE_API = "https://nomad-movies.nomadcoders.workers.dev"
MODEL = "gpt-4o-mini"

client = openai.OpenAI()

In [4]:
def get_popular_movies():
    movies = requests.get(f"{MOVIE_API}/movies").json()
    return movies

def get_movie_details(movie_id):
    movie_detail = requests.get(f"{MOVIE_API}/movies/{movie_id}").json()
    return movie_detail

def get_movie_credits(movie_id):
    movie_credits = requests.get(f"{MOVIE_API}/movies/{movie_id}/credits").json()
    return movie_credits

In [5]:
FUNCTION_MAP = {"get_popular_movies": get_popular_movies, "get_movie_details": get_movie_details, "get_movie_credits": get_movie_credits}

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_popular_movies",
            "description": "인기 영화 목록을 조회한다",
            "parameters": {"type": "object", "properties": {}}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_details",
            "description": "영화 ID로 상세 정보를 조회한다",
            "parameters": {
                "type": "object",
                "properties": {
                    "movie_id": {
                        "type": "integer",
                        "description": "영화 ID"
                    }
                },
                "required": ["movie_id"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_movie_credits",
            "description": "영화 ID로 출연진(캐스트, 크루) 정보를 조회한다",
            "parameters": {
                "type": "object",
                "properties": {
                    "movie_id": {
                        "type": "integer",
                        "description": "영화 ID"
                    }
                },
                "required": ["movie_id"]
            }
        }
    }
]

In [10]:

def process_ai_response(message: ChatCompletionMessage, messages):
    if message.tool_calls:
        messages.append({
            "role": "assistant",
            "content": message.content or "",
            "tool_calls": [
                {
                    "id": tool_call.id,
                    "type": "function",
                    "function": {
                        "name": tool_call.function.name,
                        "arguments": tool_call.function.arguments
                    }
                }
                for tool_call in message.tool_calls
                ]
        })

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments
            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                print(f"Error parsing arguments: {arguments}")
                arguments = {}
            function_to_run = FUNCTION_MAP[function_name]
            result = function_to_run(**arguments)
            print(f"Ran {function_name} with args {arguments} for a result of {result}")
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "name": function_name,
                "content": json.dumps(result, ensure_ascii=False)
            })
        call_ai(messages)

    else:
        messages.append({
            "role": "assistant",
            "content": message.content
        })
        print(f"Assistant: {message.content}")

    return messages

def call_ai(messages):
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=TOOLS
    )
    messages = process_ai_response(response.choices[0].message, messages)
    return messages



In [12]:
messages = []

while True:
    message = input("Send a message to the LLM...")
    if message == "quit" or message == "q":
        break
    else:
        messages.append({
            "role": "user",
            "content": message})
        print(f"User: {message}")
        messages = call_ai(messages)


User: 지금 인기있는 영화가 무엇인지 알려줘
Ran get_popular_movies with args {} for a result of [{'adult': False, 'backdrop_path': 'https://image.tmdb.org/t/p/w1280/7HKpc11uQfxnw0Y8tRUYn1fsKqE.jpg', 'genre_ids': [878, 28, 53], 'id': 1236153, 'original_language': 'en', 'original_title': 'Mercy', 'overview': 'In the near future, a detective stands on trial accused of murdering his wife. He has ninety minutes to prove his innocence to the advanced AI Judge he once championed, before it determines his fate.', 'popularity': 649.4032, 'poster_path': 'https://image.tmdb.org/t/p/w780/pyok1kZJCfyuFapYXzHcy7BLlQa.jpg', 'release_date': '2026-01-20', 'title': 'Mercy', 'video': False, 'vote_average': 7.054, 'vote_count': 468}, {'adult': False, 'backdrop_path': 'https://image.tmdb.org/t/p/w1280/hHDNOlATHhre4eZ7aYz5cdyJLik.jpg', 'genre_ids': [27, 53, 878], 'id': 1272837, 'original_language': 'en', 'original_title': '28 Years Later: The Bone Temple', 'overview': "Dr. Kelson finds himself in a shocking new relationship